In [ ]:
import pandas as pd, numpy as np
import statsmodels.api as sm
from scipy import stats

In [2]:


# 读取
df = pd.read_excel(r"G:\25国赛\C题\PythonProject\code4\筛.xlsx")

y_col = "abnormal"
x_cols = ["年龄","检测孕周","GC含量","原始读段数","在参考基因组上比对的比例",
          "13号染色体的Z值","18号染色体的Z值","21号染色体的Z值","X染色体的Z值",
          "X染色体浓度","13号染色体的GC含量","18号染色体的GC含量","21号染色体的GC含量"]

# 选取并转数值、去缺失
df_use = df[[y_col] + x_cols].copy()
for c in df_use.columns:
    df_use[c] = pd.to_numeric(df_use[c], errors="coerce")
df_use = df_use.dropna().reset_index(drop=True)

X = sm.add_constant(df_use[x_cols])
y = df_use[y_col]
model = sm.Logit(y, X)
res = model.fit(disp=False)


df_use["p_hat"] = res.predict(X)

# Hosmer-Lemeshow检验（按预测概率分位分组）
def hosmer_lemeshow(y_true, p_pred, g=10):
    data = pd.DataFrame({"y": y_true, "p": p_pred}).sort_values("p")
    data["group"] = pd.qcut(data["p"], q=g, duplicates="drop")
    tab = data.groupby("group").agg(n=("y","size"),
                                    y_obs=("y","sum"),
                                    p_bar=("p","mean")).reset_index()
    tab["y_exp"] = tab["n"]*tab["p_bar"]
    chi2 = np.sum((tab["y_obs"] - tab["y_exp"])**2 /
                  (tab["n"]*tab["p_bar"]*(1-tab["p_bar"]) + 1e-12))
    dof = tab.shape[0] - 2
    pval = 1 - stats.chi2.cdf(chi2, dof)
    return tab, chi2, dof, pval

hl_table, hl_chi2, hl_df, hl_p = hosmer_lemeshow(y, df_use["p_hat"], g=10)

params = res.params
conf = res.conf_int()
or_table = pd.DataFrame({
    "变量": params.index,
    "系数": params.values,
    "OR(=exp(系数))": np.exp(params.values),
    "CI下限(OR)": np.exp(conf[0].values),
    "CI上限(OR)": np.exp(conf[1].values),
    "p值": res.pvalues.values
})

# 概率计算方程（logit → 概率）
eq_terms = [f"{coef:.6f}*{name}" for name, coef in params.items() if name!="const"]
equation = f"线性预测η = {params['const']:.6f} + " + " + ".join(eq_terms) + \
           "\n概率 p = 1 / (1 + exp(-η))"

# 保存结果
or_path = "Logit_OR表.csv"
hl_path = "HL检验分组表.csv"
sum_path = "Logit_摘要.txt"
or_table.to_csv(or_path, index=False)
hl_table.to_csv(hl_path, index=False)
with open(sum_path, "w", encoding="utf-8") as f:
    f.write(res.summary2().as_text())
    f.write("\n\n--- 概率计算方程 ---\n")
    f.write(equation)
    f.write(f"\n\n--- Hosmer-Lemeshow 检验 ---\nChi2 = {hl_chi2:.3f}, df = {hl_df}, p = {hl_p:.4f}\n")
    f.write("解释：当 p 不显著(>0.05) → 拟合效果较好；当 p 显著(≤0.05) → 拟合较差。\n")


C:\Users\SerenadeedanereS\AppData\Local\Temp\ipykernel_21960\3501223317.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tab = data.groupby("group").agg(n=("y","size"),
C:\Users\SerenadeedanereS\AppData\Local\Temp\ipykernel_21960\3501223317.py:51: RuntimeWarning: overflow encountered in exp
  "OR(=exp(系数))": np.exp(params.values),
C:\Users\SerenadeedanereS\AppData\Local\Temp\ipykernel_21960\3501223317.py:53: RuntimeWarning: overflow encountered in exp
  "CI上限(OR)": np.exp(conf[1].values),
